In [1]:
import pandas as pd
import numpy as np
import psycopg2
import random
from random import randint
from configparser import ConfigParser

# 1. Подключение к СУБД

In [2]:
def config(filename='DBtest.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [3]:
params = config()
def connect():
    conn = None
    try:
        # read connection parameters

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()
        
        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
        # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
    
    
if __name__ == '__main__':
   connect()


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.


# 2. Создание таблиц:

In [5]:
def create_tables():
    commands = ("""CREATE TABLE IF NOT EXISTS Users (
                    user_id SERIAL PRIMARY KEY,
                    age INT2 NOT NULL)""",
                
                """CREATE TABLE IF NOT EXISTS Items (
                   item_id SERIAL PRIMARY KEY,
                   price FLOAT4 NOT NULL)""",
                
                """CREATE TABLE IF NOT EXISTS Purchases (
                    purchase_id SERIAL PRIMARY KEY,
                    user_id INT2 NOT NULL,
                    FOREIGN KEY (user_id) REFERENCES Users (user_id) ON UPDATE CASCADE ON DELETE CASCADE,
                    item_id INT2 NOT NULL,
                    FOREIGN KEY (item_id) REFERENCES Items (item_id) ON UPDATE CASCADE ON DELETE CASCADE,
                    date DATE)""",
        
        )
    conn = None
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


if __name__ == '__main__':
    create_tables()

## 2.1 Функция получения заголовков таблицы

In [9]:
from psycopg2 import sql, connect



# define a function that gets the column names from a PostgreSQL table
def get_columns_names(table):
    conn = None
    try:
        # declare a new PostgreSQL connection object
        conn = psycopg2.connect(**params)
    
    # declare an empty list for the column names
        column_names = []

    # declare cursor objects from the connection    
        col_cursor = conn.cursor()

        sql = '''SELECT * FROM {}'''.format(table)
        col_cursor.execute(sql)
        column_names = [desc[0] for desc in col_cursor.description]

        # close the cursor object to prevent memory leaks
        col_cursor.close()

    except Exception as err:
        print ("get_columns_names ERROR:", err)

    # return the list of column names
    return column_names
get_columns_names('Users')

['user_id', 'age']

## 2.2 Функция вывода таблицы в датафрейм

In [26]:
def postgres_to_dataframe(select_query):
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    try:
        cur.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cur.close
        return 1
    
    column_names = [desc[0] for desc in cur.description]
    tupples = cur.fetchall()
    cur.close
    
    df = pd.DataFrame(tupples, columns = column_names)
    return df
    

## 2.3 Функция заполнения таблиц путём INSERT INTO

In [10]:
def insert(table_name, record_to_insert, x):
    try:
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        columns = get_columns_names(table_name)
        columns_str = ""
        value_str = ""
        sign = "%s"
        for i in range(len(columns)):
            if (i != len(columns) -1):
                columns_str+="{},".format(columns[i])
                value_str+="{},".format(sign)
            else:
                columns_str+=columns[i]
                value_str+=sign
        
        # execute the INSERT statement
        postgres_insert_query = f""" INSERT INTO {table_name} ({columns_str}) VALUES ({value_str})"""
        cur.execute(postgres_insert_query, record_to_insert)
        conn.commit()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        print(f"{x}. Запись успешно добавлена в таблицу {table_name}")

# 3. Заполнение таблиц данными

In [12]:
for i in range (0, 15):
    record_to_insert = (i+1, randint(18, 70))
    insert('Users', record_to_insert, i)


0. Запись успешно добавлена в таблицу Users
1. Запись успешно добавлена в таблицу Users
2. Запись успешно добавлена в таблицу Users
3. Запись успешно добавлена в таблицу Users
4. Запись успешно добавлена в таблицу Users
5. Запись успешно добавлена в таблицу Users
6. Запись успешно добавлена в таблицу Users
7. Запись успешно добавлена в таблицу Users
8. Запись успешно добавлена в таблицу Users
9. Запись успешно добавлена в таблицу Users
10. Запись успешно добавлена в таблицу Users
11. Запись успешно добавлена в таблицу Users
12. Запись успешно добавлена в таблицу Users
13. Запись успешно добавлена в таблицу Users
14. Запись успешно добавлена в таблицу Users


### Таблица Users

In [28]:
postgres_to_dataframe('''SELECT * FROM Users''')

,user_id,age
0,1,51
1,2,25
2,3,53
3,4,45
4,5,42
5,6,53
6,7,49
7,8,27
8,9,24
9,10,35


In [15]:
for i in range (0, 8):
    record_to_insert = (i+1, round(random.uniform(500, 5000), 3))
    insert('Items', record_to_insert, i)

0. Запись успешно добавлена в таблицу Items
1. Запись успешно добавлена в таблицу Items
2. Запись успешно добавлена в таблицу Items
3. Запись успешно добавлена в таблицу Items
4. Запись успешно добавлена в таблицу Items
5. Запись успешно добавлена в таблицу Items
6. Запись успешно добавлена в таблицу Items
7. Запись успешно добавлена в таблицу Items


### Таблица Items 

In [29]:
postgres_to_dataframe('''SELECT * FROM Items''')

,item_id,price
0,1,4899.348
1,2,834.692
2,3,4132.709
3,4,1935.671
4,5,1658.851
5,6,963.547
6,7,2382.050
7,8,2997.020


In [24]:
def insert_main(table_name, i):
    try:
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        columns = get_columns_names(table_name)
        columns_str = ""
        
        for i in range(0, len(columns)):
            if (i != len(columns) - 1):
                columns_str+="{},".format(columns[i])
            else:
                columns_str+=columns[i]
                
        dates = ["15.03.2022", "15.03.2022", "21.03.2022", "25.03.2022", "04.04.2022", "04.04.2022", "04.04.2022", "09.04.2022", "10.04.2022", "13.04.2022", "16.04.2022", "18.04.2022", "25.04.2022", "03.05.2022", "05.05.2022", "27.05.2022", "17.06.2022", "13.07.2022", "28.07.2022"]
        user_id = [1, 2, 3, 3, 4, 4, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 14, 15]
        item_id = [1, 2, 3, 4, 5, 6, 7, 8]
        
        # execute the INSERT statement 
        for i in range(0, 18):
            postgres_insert_query = f""" INSERT INTO {table_name} ({columns_str}) VALUES
            (
            '{i+1}',
            '{user_id[i]}',
            '{random.choice(item_id)}',
            '{dates[i]}'
            )
            """
            
            cur.execute(postgres_insert_query, i)
        
            
        conn.commit()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        print(f"Запись успешно добавлена в таблицу {table_name}")

In [25]:
insert_main('Purchases', i)

Запись успешно добавлена в таблицу Purchases


### Таблица Purchases

In [30]:
postgres_to_dataframe('''SELECT * FROM Purchases''')

,purchase_id,user_id,item_id,date
0,1,1,2,2022-03-15
1,2,2,8,2022-03-15
2,3,3,8,2022-03-21
3,4,3,8,2022-03-25
4,5,4,1,2022-04-04
5,6,4,4,2022-04-04
6,7,4,6,2022-04-04
7,8,5,7,2022-04-09
8,9,6,2,2022-04-10
9,10,7,3,2022-04-13


# 4. Задания

## А) Какую сумму в среднем в месяц тратит:

### Пользователи в возрастном диапазоне от 18 до 25 лет включительно:

In [90]:
postgres_to_dataframe("""SELECT AVG(Items.price) as average_price FROM Items
                         JOIN Purchases ON Items.item_id = Purchases.item_id 
                         JOIN Users ON Users.user_id = Purchases.user_id
                         WHERE Users.age > 17 
                         AND Users.age < 26""")

,average_price
0,3472.602051


### Пользователи в возрастном диапазоне от 26 до 35 лет включительно:

In [40]:
postgres_to_dataframe("""SELECT AVG(Items.price) as average_price FROM Items
                         JOIN Purchases ON Items.item_id = Purchases.item_id 
                         JOIN Users ON Users.user_id = Purchases.user_id
                         WHERE Users.age > 25 
                         AND Users.age < 36""")

,average_price
0,2910.353394


## Б) В каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая 

In [136]:
postgres_to_dataframe("""SELECT to_char(date,'Mon') as mon 
                                FROM Purchases
                         
                         JOIN Users ON Purchases.user_id = Users.user_id
                         JOIN Items on Purchases.item_id = Items.item_id
                         WHERE age > 34
                         GROUP BY mon
                         HAVING sum(price) >= ALL(SELECT sum(price) FROM Items 
                                              JOIN Purchases ON Items.item_id = Purchases.item_id
                                              JOIN Users ON Purchases.user_id = Users.user_id
                                              WHERE age > 34
                                              GROUP BY to_char(date,'Mon'))
                         """) 

,mon
0,Apr


## В) Какой товар обеспечиает наибольший вклад в выручку за последний год 

In [149]:
postgres_to_dataframe("""SELECT purchases.item_id, sum(price) as total FROM Purchases, Items
                         WHERE Purchases.item_id = Items.item_id
                         AND extract(year from date) = 2022
                         GROUP BY purchases.item_id
                         HAVING sum(price) >= ALL(SELECT sum(price) FROM Purchases, Items 
                                              WHERE Purchases.item_id = Items.item_id
                                              AND extract(year from date) = 2022
                                              GROUP BY purchases.item_id)
""")

,item_id,total
0,8,17982.12


## Г) Топ-3 товаров по выручке и их доля в общей выручке за любой год

In [164]:
postgres_to_dataframe("""SELECT purchases.item_id, sum(price) as total FROM Purchases, Items
                         WHERE Purchases.item_id = Items.item_id
                         AND extract(year from date) = 2022
                         GROUP BY purchases.item_id
                         ORDER BY sum(price) DESC
                         LIMIT 3
""")

,item_id,total
0,8,17982.1200
1,1,14698.0450
2,7,7146.1504
